# BlemishBot

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

import torch
from PIL import Image
from RealESRGAN import RealESRGAN

from ultralytics import YOLO

## Load the models

In [ ]:
# Load YuNet face detection model
IMG_SIZE = 320
yunet_path = 'models/face_detection_yunet_2023mar.onnx'
yunet_face_detector = cv2.FaceDetectorYN_create(yunet_path, "", (IMG_SIZE, IMG_SIZE), score_threshold=0.5)

# Load the YOLOv8 acne detection model
yolo_path = 'models/acne_model.pt'
yolo_acne_model = YOLO(yolo_path)

# Load upscaling model
# sr_model = dnn_superres.DnnSuperResImpl_create()
# sr_model_path = 'SuperResDNN_x64/dnn_Models/FSRCNN_x4.pb'
# sr_model.readModel(sr_model_path)
# sr_model.setModel('fsrcnn', 4)

device = torch.device('cpu')
sr_model = RealESRGAN(device, scale=4)
sr_model.load_weights('models/RealESRGAN_x4.pth', download=True)

# Load classification model



In [ ]:
# test_image_path = 'detection_test.webp'
test_image_path = 'sample_inputs/detection_test_3.webp'
test_image = cv2.imread(test_image_path)

# Upscale an image
# test_image = Image.open(test_image_path).convert("RGB")
# upscaled_image = model.predict(test_image)
# upscaled_image.save("upscaled_image.jpg")

plt.imshow(test_image)
plt.axis('off')


## Prepare Image
- Initializes an image from an image_path string
- Returns the RGB image

In [ ]:
def prepare_image(image_path):
  image = cv2.imread(image_path)
  # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  # Create a square padding
  h, w = image.shape[:2]
  new_size = max(h, w)
  
  padded_image = np.full((new_size, new_size, 3), (0, 0, 0), dtype=np.uint8)

  # Calculate the position to place the original image
  x_offset = (new_size - w) // 2
  y_offset = (new_size - h) // 2

  # Place the original image in the center
  padded_image[y_offset:y_offset + h, x_offset:x_offset + w] = image

  return padded_image

In [ ]:
test_image = prepare_image(test_image_path)
plt.imshow(test_image)
plt.axis('off')

## Face detection
- Resize an image for the YuNet model
- Check if only one face is detected
- Returns a cropped image, where the face is isolated from the background

In [ ]:
def face_detection_crop(image):
  # resize image
  resized_image = cv2.resize(image, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_CUBIC)

  # detect face with YuNet
  faces = yunet_face_detector.detect(resized_image)
  
  # assert that faces were detected
  if faces[1] is None:
    print("No face detected.")
    return None
  
  # assert that only one face is detected
  elif len(faces[1]) > 1:
    print("There can only be one face in the image")
    return None
  
  # crop the face
  for face in faces[1]:
    x,y,w,h = int(face[0]), int(face[1]), int(face[2]), int(face[3])

    # ensure boundary boxes are within the image dimensions
    x = max(0, x)
    y = max(0, y)
    w = min(w, IMG_SIZE - x)
    h = min(h, IMG_SIZE - y)

    cropped_image = resized_image[y:y+h, x:x+w]

    return cropped_image


In [ ]:
# testing face_detection_crop function
new_test_image = face_detection_crop(test_image)

fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].imshow(test_image)
axs[0].set_title('Before')
axs[0].axis('off')

axs[1].imshow(new_test_image)
axs[1].set_title('After')
axs[1].axis('off')

plt.tight_layout()


## Acne detection
- detect acne regions
- return acne regions

In [ ]:
def acne_detection(image):
  results = yolo_acne_model.predict(image, conf=0.3)

  h,w = image.shape[:2]

  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  outputs = []
  offset = 50
  for result in results:
    for box in result.boxes:
      x1, y1, x2, y2 = map(int, box.xyxy[0])
      x1-=offset
      x1 = max(0, x1)

      x2+=offset
      x2 = min(x2, w)

      y1-=offset
      y1 = max(0, y1)

      y2+=offset
      y2 = min(y2, h)

      acne_region = image[y1:y2, x1:x2]
      # sharpen_kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
      # acne_region = cv2.filter2D(acne_region, -1, sharpen_kernel)

      acne_region = sr_model.predict(acne_region)

      outputs.append(acne_region)

  return outputs


In [ ]:
def acne_detection_rect(image):
  results = yolo_acne_model.predict(image, conf=0.3, iou=0.3, max_det=10)

  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  offset = 10
  for result in results:
    for box in result.boxes:
      x1, y1, x2, y2 = map(int, box.xyxy[0])
      x1-=offset
      x2+=offset
      y1-=offset
      y2+=offset
      
      cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

  return image

In [ ]:
result = acne_detection_rect(new_test_image)
plt.imshow(result)
plt.axis('off')

In [ ]:
# Usage
outputs = acne_detection(new_test_image)
k = 0

nrows = int(len(outputs)/3) + 1
ncols = 3
fig, axs = plt.subplots(nrows, ncols, figsize=(9, 3*nrows))

for i in range(0, nrows):
  for j in range(0, ncols):
    if k < len(outputs):
      axs[i][j].imshow(outputs[k])
      axs[i][j].axis('off')
      k+=1
    else:
      axs[i][j].axis('off')

plt.tight_layout()


# YOLOv8 Model

In [ ]:
result = cv2.imread('datasets/kaggle-acne/test/images/acne-5_jpeg.rf.2d6671715f0149df7b494c4d3f12a98b.jpg')
check = yolo_acne_model.predict(result, conf=0.3)
check[0].boxes

for r in check:
    boxes = r.boxes  # Bounding boxes
    for box in boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Get bounding box coordinates
        
        # Optionally, draw bounding boxes on the image or process further
        cv2.rectangle(result, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)

result = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
plt.imshow(result)
plt.axis('off')